In [99]:
import numpy as np
import requests
import pandas as pd
import scipy
import json
import urllib
import os
import xmltodict

In [61]:
musicBrainzURL =  "https://musicbrainz.org/ws/2/"
transcription_data = pd.read_csv("../../RollingStone/RollingStoneSampleData.tsv", sep="\t")
# sample_data = pd.read_csv("../BillboardSampleData.tsv", sep="\t")

In [66]:
len(transcription_data)

200

In [64]:
path = "../../RollingStone/Humdrum/"
ext = ".hum"
files = os.listdir(path)
file_names = [os.path.splitext(file)[0] for file in files if os.path.splitext(file)[1] == ext]
file_names.sort()

In [65]:
len(file_names)

200

In [100]:
r = requests.get(url = "https://musicbrainz.org/ws/2/recording/64da7cb2-f164-4a92-b1a7-5db10d445927")
r.raise_for_status()
o = xmltodict.parse(r.content)
json.dumps(o)

'{"metadata": {"@xmlns": "http://musicbrainz.org/ns/mmd-2.0#", "recording": {"@id": "64da7cb2-f164-4a92-b1a7-5db10d445927", "title": "(We\'re Gonna) Rock Around the Clock", "length": "132066", "first-release-date": "1989"}}}'

In [112]:
r.content

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><metadata created="2023-10-13T17:11:15.399Z" xmlns="http://musicbrainz.org/ns/mmd-2.0#" xmlns:ns2="http://musicbrainz.org/ns/ext#-2.0"><recording-list count="48" offset="0"><recording id="1e6f4c82-391b-435f-ba51-50c825389c34" ns2:score="100"><title>Honey Honey</title><artist-credit><name-credit><name>ABBA</name><artist id="d87e52c5-bb8d-4da8-b941-9f4928627dc8"><name>ABBA</name><sort-name>ABBA</sort-name><disambiguation>Swedish pop group</disambiguation><alias-list><alias sort-name="\xe1\x97\x85\xe1\x97\xba\xe1\x97\xb7\xe1\x97\x85" type="Search hint" type-id="1937e404-b981-3cb7-8151-4c86ebfc8d8e">\xe1\x97\x85\xe1\x97\xba\xe1\x97\xb7\xe1\x97\x85</alias><alias sort-name="Abba">Abba</alias><alias sort-name="Bj\xc3\xb6rn + Benny + Anna + Frieda">Bj\xc3\xb6rn + Benny + Anna + Frieda</alias><alias sort-name="Bj\xc3\xb6rn Benny Agnetha &amp; Anni-Frid">Bj\xc3\xb6rn Benny Agnetha &amp; Anni-Frid</alias><alias locale="bg" sort-name="ABBA" 

In [104]:
myurl = "http://musicbrainz.org/ws/2/recording/?query=artist:abba%20AND%20title:honey%20honeyAND%20country:US"
r = requests.get(url = myurl)
r.raise_for_status()
o = xmltodict.parse(r.content)
o              

OrderedDict([('metadata',
              OrderedDict([('@created', '2023-10-13T17:11:15.399Z'),
                           ('@xmlns', 'http://musicbrainz.org/ns/mmd-2.0#'),
                           ('@xmlns:ns2',
                            'http://musicbrainz.org/ns/ext#-2.0'),
                           ('recording-list',
                            OrderedDict([('@count', '48'),
                                         ('@offset', '0'),
                                         ('recording',
                                          [OrderedDict([('@id',
                                                         '1e6f4c82-391b-435f-ba51-50c825389c34'),
                                                        ('@ns2:score', '100'),
                                                        ('title',
                                                         'Honey Honey'),
                                                        ('artist-credit',
                                              

In [111]:
o["metadata"]["recording-list"]["recording"]

[OrderedDict([('@id', '1e6f4c82-391b-435f-ba51-50c825389c34'),
              ('@ns2:score', '100'),
              ('title', 'Honey Honey'),
              ('artist-credit',
               OrderedDict([('name-credit',
                             OrderedDict([('name', 'ABBA'),
                                          ('artist',
                                           OrderedDict([('@id',
                                                         'd87e52c5-bb8d-4da8-b941-9f4928627dc8'),
                                                        ('name', 'ABBA'),
                                                        ('sort-name', 'ABBA'),
                                                        ('disambiguation',
                                                         'Swedish pop group'),
                                                        ('alias-list',
                                                         OrderedDict([('alias',
                                                   

In [74]:
def lookup_metadata(track_name: str, artist_name: str, incs: str) -> dict:
    """Looks up the metadata for a listen using track name and artist name."""
    params = {
        "recording_name": track_name,
        "artist_name": artist_name,
        "type": "release"
    }
    if incs:
        params["metadata"] = True
        params["incs"] = incs
#         params["type"] = "release"
    response = requests.get(
        url="https://api.listenbrainz.org/1/metadata/lookup/",
        params=params
    )
    response.raise_for_status()
    return response.json()

In [75]:
lookup_metadata(lookups["TITLE"][1], lookups["ARTIST"][1], "")

{'artist_credit_name': 'Bill Haley and His Comets',
 'artist_mbids': ['4458d70d-c215-4f06-beea-ebd448dad6ac'],
 'recording_mbid': '64da7cb2-f164-4a92-b1a7-5db10d445927',
 'recording_name': "(We're Gonna) Rock Around the Clock",
 'release_mbid': '24ed3b37-3ad0-4efe-bc57-5073ec928149',
 'release_name': 'Rock Around the Clock'}

In [68]:
lookups = transcription_data[transcription_data["FileName"].isin(file_names)]
# lookups["mbid"] = [lookup_metadata(title, artist, "").get("recording_mbid") for title, artist in zip(lookups["TITLE"], lookups["ARTIST"])]

<ipython-input-68-fb18fc6dbdc5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookups["mbid"] = [lookup_metadata(title, artist, "").get("recording_mbid") for title, artist in zip(lookups["TITLE"], lookups["ARTIST"])]


In [69]:
len(lookups)

197

In [70]:
for i in range(len(lookups)):
    row = lookups.iloc[i]
    humfile = open(path + row["FileName"] + ext, "a")
    humfile.write("!!!MBID: "+str(row["mbid"])+"\n")
    humfile.close()